<a href="https://colab.research.google.com/github/itoyuhao/data-course-sample/blob/main/collaborative_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 載入套件

In [2]:
#!pip install surprise

     |████████████████████████████████| 11.8 MB 929 kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619430 sha256=6b4c410d964df749bbbf5eceeb62e39d839963b02f8ad82095c7ca278b7203c6
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [13]:
# load package

import pandas as pd
import numpy as np
from itertools import combinations
from collections import defaultdict
import datetime
from surprise import Reader
from surprise import Dataset
from surprise import KNNBasic

## 基礎建設

In [4]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [5]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2022-01-09 11:14:26--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv’

All_Beauty.csv      100%[===================>]  14.78M  19.1MB/s    in 0.8s    

2022-01-09 11:14:27 (19.1 MB/s) - ‘All_Beauty.csv’ saved [15499476/15499476]

--2022-01-09 11:14:27--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz’

meta_All_Beauty.jso 100%[===================>]   9.85M  15.2MB/s    in 0.6s    

202

In [6]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [7]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [8]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


In [9]:
print(metadata.shape)
print(ratings.shape)

(32892, 19)
(371345, 4)


In [10]:
ratings.describe()

,overall,unixReviewTime
count,371345.000000,3.713450e+05
mean,4.112093,1.440742e+09
std,1.362099,6.962743e+07
min,1.000000,9.474624e+08
25%,4.000000,1.416096e+09
50%,5.000000,1.456963e+09
75%,5.000000,1.485562e+09
max,5.000000,1.538438e+09


## 資料整理

In [11]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

In [14]:
# 去除重複的資料
metadata = metadata.loc[metadata.astype(str).drop_duplicates().index]
metadata = metadata.reset_index()
# 以np.NaN取得空值
metadata.replace('', np.NaN, inplace = True)

## 資料切分

In [15]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

## 產生推薦

In [16]:
# function of getting best selling product (Rule-Based)
def get_best_products(training_data, top=10, days_ago=30):
    # 取近 N 天的銷售紀錄 (params:days_ago)
    training_data = training_data[(training_data['DATE'] + datetime.timedelta(days=days_ago)) > '2018-09-01']
    # 以銷售量與平均評分作為依據
    best_selling = training_data.groupby('asin').agg({'overall':['mean', 'size']})['overall'].reset_index()
    # 以銷售量為主, 平均評分為輔來排序，得出排名
    best_selling = best_selling.sort_values(['size', 'mean'], ascending=False)
    # 根據需要，取足夠數量的商品進行推薦
    recom_products = best_selling['asin'].tolist()[:top]
    return recom_products

In [23]:
# function of evaluating recommender (Recall)
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score
# evaluate(ratings_testings_by_user, ratings_by_user)

In [17]:
# getting user similarity matrix
def get_user_similarity(training_data, remove_obscure_user = True, user_rating_threshold=3):
    
    # loading data from dataframe
    # user_to_items dict:
    # {
    #   'user': {
    #       'item': ratings...
    #   }...
    # }
    
    # 依據時間先後把重複的資料過濾掉，如果使用者對同一商品有重複評論的話，只留下最新的一筆評分。
    print('Starting filtering data...')
    
    training_data = (
        training_data
        .sort_values('DATE', ascending=False)
        .groupby(['reviewerID', 'asin']).head(1)
    )
    
    # 先做出user_to_items的dict
    user_to_items = defaultdict(dict)
    for _, row in training_data.iterrows():
        row = dict(row)
        user = row['reviewerID']
        item = row['asin']
        rating = float(row['overall'])

        user_to_items[user][item] = rating

    # remove obscure user to decrease data size (過濾掉出現次數小於 n 的使用者)
    all_users = list(user_to_items.keys())
    for user in all_users:
        ratings = user_to_items[user]
        if remove_obscure_user and len(ratings) < user_rating_threshold:
            del user_to_items[user]

    
    # 透過將user_to_items轉置為item_to_users來優化運算效率
    # generate item to user mapping dict
    # {
    #   'item': {
    #       'user': ratings...
    #   }...
    # }
    item_to_users = defaultdict(dict)
    for user, items in user_to_items.items():
        for item, rating in items.items():
            item_to_users[item][user] = rating

    # prepare data of computing user similarity 
    init_sim = lambda: [0 for _ in range(3)]
    factory = lambda: defaultdict(init_sim)
    pre_user_similarity = defaultdict(factory)
    n = len(item_to_users)
    index = 0
    for item, user_ratings in item_to_users.items():
        if len(user_ratings) > 1:
            # print(f"item: {item} have been rated by {len(user_ratings)} users progress: {index}/{n}")
            for user1, user2 in combinations(user_ratings.keys(), 2):
                xy = user_ratings[user1] * user_ratings[user2]
                xx = user_ratings[user1] ** 2
                yy = user_ratings[user2] ** 2
                pre_user_similarity[user1][user2][0] += xy
                pre_user_similarity[user1][user2][1] += xx
                pre_user_similarity[user1][user2][2] += yy

                pre_user_similarity[user2][user1][0] += xy
                pre_user_similarity[user2][user1][1] += xx
                pre_user_similarity[user2][user1][2] += yy
        index += 1
    
    print("Start calculating user similarity...")
    user_similarity = {}
    for src_user in pre_user_similarity:
        user_similarity_order = []
        for dst_user, val in pre_user_similarity[src_user].items():
            xy = val[0]
            xx = val[1]
            yy = val[2]
            div = ((xx*yy) ** 0.5)
            if div == 0:
                continue
            similarity = xy / div
            if similarity < 0:
                continue
            for i, s in enumerate(user_similarity_order):
                target_similarity = s[1]
                if target_similarity < similarity:
                    user_similarity_order.insert(i, (dst_user, similarity))
                    break
            else:
                user_similarity_order.append((dst_user, similarity))
        user_similarity[src_user] = user_similarity_order # {src_user:[(dst_user, similarity), (dst_user, similarity)]}
    
    return user_similarity, user_to_items

In [18]:
def user_based_recommender(training_data, user_similarity, user_to_items, users=[], days_ago=30, k=10):

    recommendation = {}
    for user in users:
        # 以下是user_based的推薦法
        if user in user_similarity:
            sim_users = user_similarity[user]
            recommended_items = []
            recommended_items_set = set()
            user_have_rated = set(user_to_items[user])
            stop_recommend = False
            for sim_user, _ in sim_users:
                items_from_sim_user = sorted(list(user_to_items[sim_user].items()), key=lambda item: item[1])
                for item, _ in items_from_sim_user:
                    if item not in user_have_rated and item not in user_have_rated:
                        recommended_items.append(item)
                        recommended_items_set.add(item)
                    if len(recommended_items) >= k:
                        stop_recommend = True
                        break
                if stop_recommend:
                    break
            recommendation[user] = recommended_items
        else:
            recommendation[user] = []
        # 以下是rule_based推薦法(當user_based推薦數不足時得補足)
        if len(recommendation[user]) < k:
            n = k - len(recommendation[user])
            recom_items = get_best_products(training_data, top=n, days_ago=days_ago)
            recommendation[user].extend(recom_items)
    return recommendation

# ratings_by_user = user_based_recommender(ratings_trainings, similarity_dict, user_to_items_dict, users, days_ago=15)
# ratings_by_user

## User-based 結果評估

In [19]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score
# evaluate(ratings_testings_by_user, ratings_by_user)

In [20]:
# 嘗試用各種不同的時間區段、推薦數量來檢驗推薦效果
days = [10, 30, 60, 90, 180]
recoms = [5, 10, 20, 50]
similarity_dict, user_to_items_dict = get_user_similarity(ratings_trainings)

print('User-based & Rule-based混合推薦法')
print('結果如下：')
for d in days:
    print('---------------------------------------------------------------------------------------------------------------')
    for recom in recoms:
        ratings_by_user = user_based_recommender(ratings_trainings, similarity_dict, user_to_items_dict, users, days_ago=d, k=recom)
        result = round(evaluate(ratings_testings_by_user, ratings_by_user), 5)
        print(f'以User-based CF推薦使用者 {recom} 個商品，若推薦數不足則以近 {d} 天的熱銷商品進行推薦，評估分數(Recall)：{result}')


Starting filtering data...
Start calculating user similarity...
User-based & Rule-based混合推薦法
結果如下：
---------------------------------------------------------------------------------------------------------------
以User-based CF推薦使用者 5 個商品，若推薦數不足則以近 10 天的熱銷商品進行推薦，評估分數(Recall)：0.12203
以User-based CF推薦使用者 10 個商品，若推薦數不足則以近 10 天的熱銷商品進行推薦，評估分數(Recall)：0.14407
以User-based CF推薦使用者 20 個商品，若推薦數不足則以近 10 天的熱銷商品進行推薦，評估分數(Recall)：0.22034
以User-based CF推薦使用者 50 個商品，若推薦數不足則以近 10 天的熱銷商品進行推薦，評估分數(Recall)：0.26949
---------------------------------------------------------------------------------------------------------------
以User-based CF推薦使用者 5 個商品，若推薦數不足則以近 30 天的熱銷商品進行推薦，評估分數(Recall)：0.08814
以User-based CF推薦使用者 10 個商品，若推薦數不足則以近 30 天的熱銷商品進行推薦，評估分數(Recall)：0.15763
以User-based CF推薦使用者 20 個商品，若推薦數不足則以近 30 天的熱銷商品進行推薦，評估分數(Recall)：0.19322
以User-based CF推薦使用者 50 個商品，若推薦數不足則以近 30 天的熱銷商品進行推薦，評估分數(Recall)：0.34237
-------------------------------------------------------------------------------------------------------

## Item-based 推薦法

In [21]:
# getting item similarity matrix
def get_item_similarity(training_data):
    
    # loading data from dataframe
    # item_to_users dict:
    # {
    #   'item': {
    #       'user': ratings...
    #   }...
    # }
    
    print('Start processing data...')
    # 與user-based的做法相同，依據時間先後把重複的資料過濾掉，如果使用者對同一商品有重複評論的話，只留下最新的一筆評分
    training_data = (
        training_data
        .sort_values('DATE', ascending=False)
        .groupby(['reviewerID', 'asin']).head(1)
    )
    
    # 透過將item_to_users轉置為user_to_items來優化運算效率
    item_to_users = defaultdict(dict)
    for _, row in training_data.iterrows():
        row = dict(row)
        user = row['reviewerID']
        item = row['asin']
        rating = float(row['overall'])
        item_to_users[item][user] = rating

    
    user_to_items = defaultdict(dict)
    for item, rating_users in item_to_users.items():
        for user, rating in rating_users.items():
            user_to_items[user][item] = rating

    print("Start calculating item similarity...")
    init_sim = lambda: [0, 0, 0]
    factory = lambda: defaultdict(init_sim)
    pre_item_similarity = defaultdict(factory)
    for user, items in user_to_items.items():
        if len(items) > 1:
            for i1, i2 in combinations(items.keys(), 2):
                xy = items[i1] * items[i2]
                xx = items[i1] ** 2
                yy = items[i2] ** 2
                pre_item_similarity[i1][i2][0] += xy
                pre_item_similarity[i1][i2][1] += xx
                pre_item_similarity[i1][i2][2] += yy

                pre_item_similarity[i2][i1][0] += xy
                pre_item_similarity[i2][i1][1] += xx
                pre_item_similarity[i2][i1][2] += yy


    item_similarity = {}
    for src_item in pre_item_similarity:
        item_similarity_order = []
        for dst_item, val in pre_item_similarity[src_item].items():
            xy = val[0]
            xx = val[1]
            yy = val[2]
            div = ((xx*yy) ** 0.5)
            if div == 0:
                continue
            similarity = xy / div
            if similarity < 0:
                continue
            for i, s in enumerate(item_similarity_order):
                target_similarity = s[1]
                if target_similarity < similarity:
                    item_similarity_order.insert(i, (dst_item, similarity))
                    break
            else:
                item_similarity_order.append((dst_item, similarity))
        item_similarity[src_item] = item_similarity_order
    return item_similarity, user_to_items

In [22]:
def item_based_recommender(training_data, item_similarity, user_to_items, users=[], days_ago=30, k=10):
    
    recommendation = {}
    for user in users:
        items = []
        items_set = set()
        stop = False
        user_has_rated = set(user_to_items[user])
        # 以下是item-based推薦法
        for item in user_has_rated:
            if item in item_similarity:
                for sim_item, _ in item_similarity[item]:
                    # skip the item user has rated
                    if sim_item not in user_has_rated and sim_item not in items_set:
                        items.append(sim_item)
                        items_set.add(sim_item)
                    if len(items) >= k:
                        stop = True
                        break
                if stop:
                    break
            
        recommendation[user] = items
        
        # 以下是rule_based推薦法(當user_based推薦數不足時得補足)
        if len(recommendation[user]) < k:
            n = k - len(recommendation[user])
            recom_items = get_best_products(training_data, top=n, days_ago=days_ago)
            recommendation[user].extend(recom_items)
    return recommendation

## Item-based 結果評估

In [24]:
# 嘗試用各種不同的時間區段、推薦數量來檢驗推薦效果
days = [10, 30, 60, 90, 180]
recoms = [5, 10, 20, 50]
similarity_dict, user_to_items_dict = get_item_similarity(ratings_trainings)

print('Item-based & Rule-based混合推薦法')
print('結果如下：')
for d in days:
    print('---------------------------------------------------------------------------------------------------------------')
    for recom in recoms:
        ratings_by_user = item_based_recommender(ratings_trainings, similarity_dict, user_to_items_dict, users, days_ago=d, k=recom)
        result = round(evaluate(ratings_testings_by_user, ratings_by_user), 5)
        print(f'以Item-based CF推薦使用者 {recom} 個商品，若推薦數不足則以近 {d} 天的熱銷商品進行推薦，評估分數(Recall)：{result}')


Start processing data...
Start calculating item similarity...
Item-based & Rule-based混合推薦法
結果如下：
---------------------------------------------------------------------------------------------------------------
以Item-based CF推薦使用者 5 個商品，若推薦數不足則以近 10 天的熱銷商品進行推薦，評估分數(Recall)：0.12203
以Item-based CF推薦使用者 10 個商品，若推薦數不足則以近 10 天的熱銷商品進行推薦，評估分數(Recall)：0.14068
以Item-based CF推薦使用者 20 個商品，若推薦數不足則以近 10 天的熱銷商品進行推薦，評估分數(Recall)：0.21356
以Item-based CF推薦使用者 50 個商品，若推薦數不足則以近 10 天的熱銷商品進行推薦，評估分數(Recall)：0.2661
---------------------------------------------------------------------------------------------------------------
以Item-based CF推薦使用者 5 個商品，若推薦數不足則以近 30 天的熱銷商品進行推薦，評估分數(Recall)：0.08983
以Item-based CF推薦使用者 10 個商品，若推薦數不足則以近 30 天的熱銷商品進行推薦，評估分數(Recall)：0.15593
以Item-based CF推薦使用者 20 個商品，若推薦數不足則以近 30 天的熱銷商品進行推薦，評估分數(Recall)：0.18644
以Item-based CF推薦使用者 50 個商品，若推薦數不足則以近 30 天的熱銷商品進行推薦，評估分數(Recall)：0.3322
-----------------------------------------------------------------------------------------------------------

## 使用 surprise 進行推薦

In [25]:
def surprise_recommender(training_data, users=[], days_ago=30, k=10, user_based=False, algo=KNNBasic):

    training_data = (
        training_data
        .sort_values("DATE", ascending=False)
        .groupby(['reviewerID', 'asin']).head(1)
    )

    reader = Reader(rating_scale=(0, 5))
    surprise_data = training_data[['reviewerID', 'asin', 'overall']]
    data = Dataset.load_from_df(surprise_data, reader=reader)

    sim_options = {
        'name': 'cosine',
        'user_based': user_based  # compute similarities between items
    }
    algo_impl = algo(sim_options=sim_options)
    trainset = data.build_full_trainset()
    algo_impl.fit(trainset)

    recommendation = {}
    for user in users:
        items_user_rated = set(surprise_data.loc[surprise_data['reviewerID'] == user]['asin'].to_list())
        recommend_item_list = []
        recommend_item_set = set()
        for item in items_user_rated:
            iid = algo_impl.trainset.to_inner_iid(item)
            recommend_items_iid = algo_impl.get_neighbors(iid, k)
            for sim_item_iid in recommend_items_iid:
                item_raw_id = algo_impl.trainset.to_raw_iid(sim_item_iid)
                if item_raw_id not in items_user_rated and item_raw_id not in recommend_item_set:
                    recommend_item_list.append(item_raw_id)
                    recommend_item_set.add(item_raw_id)

            if len(recommend_item_list) >= k:
                recommend_item_list = recommend_item_list[:k]
                break
        recommendation[user] = recommend_item_list
        
        # 以下是rule_based推薦法(當user_based推薦數不足時得補足)
        if len(recommendation[user]) < k:
            n = k - len(recommendation[user])
            recom_items = get_best_products(training_data, top=n, days_ago=days_ago)
            recommendation[user].extend(recom_items)

    return recommendation

## surprise 結果評估

In [1]:
# 嘗試用各種不同的時間區段、推薦數量來檢驗推薦效果
days = [10, 30, 60, 90, 180]
recoms = [5, 10, 20, 50]

print('利用Surprise套件中KNNBasic的Item-based & Rule-based混合推薦法')
print('結果如下：')
for d in days:
    print('---------------------------------------------------------------------------------------------------------------')
    for recom in recoms:
        ratings_by_user = surprise_recommender(ratings_trainings, users, days_ago=d, k=recom)
        result = round(evaluate(ratings_testings_by_user, ratings_by_user), 5)
        print(f'以Item-based CF推薦使用者 {recom} 個商品，若推薦數不足則以近 {d} 天的熱銷商品進行推薦，評估分數(Recall)：{result}')
